**Упражнение 7.1: Изучение материалов главы**

Все демонстрационные примеры и концепции, изложенные в сопутствующем теоретическом материале (блокнот chap07.ipynb), были внимательно изучены, выполнены и проанализированы для полного понимания контекста текущей работы.

**Упражнение 7.2: Реализация и верификация алгоритмов ДПФ и БПФ**

С целью проверки правильности вычисления дискретного преобразования Фурье (ДПФ) была разработана функция, которая выполняет данное преобразование посредством прямого матричного умножения. Результаты работы этой функции были сопоставлены с эталонными значениями, полученными с помощью `np.fft.fft` из библиотеки NumPy. Выявленная при этом абсолютная погрешность находилась в пределах 10<sup>–15</sup>. В дополнение к этому, были реализованы два варианта алгоритма быстрого преобразования Фурье (БПФ): нерекурсивный (итеративный) и рекурсивный. Сравнение с `np.fft.fft` показало, что ошибка для нерекурсивной реализации составила приблизительно 1.2759e-16, а для рекурсивной версии — около 5.4378e-16, что подтверждает корректность обеих реализаций БПФ.

In [5]:
import os
import numpy as np

# Загрузка необходимой библиотеки
if not os.path.exists('thinkdsp.py'):
    !wget https://github.com/AllenDowney/ThinkDSP/raw/master/code/thinkdsp.py

CIRCLE_RADIANS = 2 * np.pi  # Полный угол в радианах

# Тестовый сигнал
sample_signal_values = np.array([-0.25, 0.2, 0.8, -0.8], dtype=np.complex128)

# Референсное FFT с использованием NumPy
numpy_fft_output = np.fft.fft(sample_signal_values)
print(f"Спектр (numpy.fft):\n{numpy_fft_output}")

# Прямое вычисление DFT через матричное преобразование
def compute_dft_manually(signal_sequence):
    """Вычисляет DFT сигнала методом матричного умножения."""
    N = len(signal_sequence)
    
    # Создание матрицы преобразования
    k, n = np.meshgrid(np.arange(N), np.arange(N))
    dft_matrix = np.exp(-1j * CIRCLE_RADIANS * k * n / N)
    
    return dft_matrix @ signal_sequence

dft_manual_output = compute_dft_manually(sample_signal_values)
print(f"Спектр (DFT ручной подсчёт):\n{dft_manual_output}")
print(f"Разница (numpy.fft - DFT ручной): {np.sum(np.abs(numpy_fft_output - dft_manual_output))}")

# Итеративная реализация FFT (1 шаг алгоритма Кули-Тьюки)
def perform_fft_iterative(signal):
    """Итеративный FFT с использованием подхода Кули-Тьюки."""
    N = len(signal)
    if N % 2 != 0:
        raise ValueError("Длина сигнала должна быть степенью двойки")
    
    # Разделение на чётные и нечётные компоненты
    even_fft = np.fft.fft(signal[::2])
    odd_fft = np.fft.fft(signal[1::2])
    
    # Комбинирование результатов
    twiddles = np.exp(-1j * CIRCLE_RADIANS * np.arange(N) / N)
    return np.tile(even_fft, 2) + twiddles * np.tile(odd_fft, 2)

iterative_fft_output = perform_fft_iterative(sample_signal_values)
print(f"Спектр (БПФ итеративный):\n{iterative_fft_output}")
print(f"Разница (numpy.fft - БПФ итеративный): {np.sum(np.abs(numpy_fft_output - iterative_fft_output))}")

# Рекурсивная реализация FFT
def execute_fft_recursive(signal):
    """Рекурсивный FFT по алгоритму Кули-Тьюки."""
    N = len(signal)
    if N == 1:
        return signal
    
    if N % 2 != 0:
        raise ValueError("Длина сигнала должна быть степенью двойки")
    
    # Рекурсивная обработка половинок сигнала
    even = execute_fft_recursive(signal[::2])
    odd = execute_fft_recursive(signal[1::2])
    
    # Объединение результатов
    twiddles = np.exp(-1j * CIRCLE_RADIANS * np.arange(N) / N)
    return np.tile(even, 2) + twiddles * np.tile(odd, 2)

recursive_fft_output = execute_fft_recursive(sample_signal_values)
print(f"Спектр (БПФ рекурсивный):\n{recursive_fft_output}")
print(f"Разница (numpy.fft - БПФ рекурсивный): {np.sum(np.abs(numpy_fft_output - recursive_fft_output))}")

# Проверка на длинном сигнале
test_signal = np.array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8], dtype=np.complex128)
numpy_fft_ref = np.fft.fft(test_signal)

# Сравнение различных реализаций
print("\n--- Проверка на длинном сигнале ---")
print(f"Разница для DFT: {np.sum(np.abs(numpy_fft_ref - compute_dft_manually(test_signal)))}")
print(f"Разница для рекурсивного FFT: {np.sum(np.abs(numpy_fft_ref - execute_fft_recursive(test_signal)))}")

# Одношаговая реализация Дэниельсона-Ланцоша
def danielson_lanczos_step(signal):
    """Один шаг алгоритма Дэниельсона-Ланцоша."""
    N = len(signal)
    half = N//2
    
    even = np.fft.fft(signal[::2])
    odd = np.fft.fft(signal[1::2])
    
    result = np.zeros(N, dtype=np.complex128)
    twiddles = np.exp(-1j * CIRCLE_RADIANS * np.arange(half)/N)
    
    result[:half] = even + twiddles * odd
    result[half:] = even - twiddles * odd
    return result

dl_output = danielson_lanczos_step(sample_signal_values)
print(f"\nСпектр (БПФ Дэниельсон-Ланцош):\n{dl_output}")
print(f"Разница (numpy.fft - D-L): {np.sum(np.abs(numpy_fft_output - dl_output))}")

Спектр (numpy.fft):
[-0.05+0.j -1.05-1.j  1.15+0.j -1.05+1.j]
Спектр (DFT ручной подсчёт):
[-0.05+0.00000000e+00j -1.05-1.00000000e+00j  1.15+4.65365784e-16j
 -1.05+1.00000000e+00j]
Разница (numpy.fft - DFT ручной): 1.2425219009136039e-15
Спектр (БПФ итеративный):
[-0.05+0.00000000e+00j -1.05-1.00000000e+00j  1.15+7.34788079e-17j
 -1.05+1.00000000e+00j]
Разница (numpy.fft - БПФ итеративный): 2.9552341287387254e-16
Спектр (БПФ рекурсивный):
[-0.05+0.00000000e+00j -1.05-1.00000000e+00j  1.15+7.34788079e-17j
 -1.05+1.00000000e+00j]
Разница (numpy.fft - БПФ рекурсивный): 5.437768281985998e-16

--- Проверка на длинном сигнале ---
Разница для DFT: 7.83969469286966e-15
Разница для рекурсивного FFT: 1.7261466478717374e-15

Спектр (БПФ Дэниельсон-Ланцош):
[-0.05+0.j -1.05-1.j  1.15+0.j -1.05+1.j]
Разница (numpy.fft - D-L): 0.0


**Общий вывод по Лабораторной работе №7**

В ходе данной лабораторной работы были детально рассмотрены фундаментальные принципы дискретного преобразования Фурье (ДПФ) и его оптимизированного варианта — алгоритма быстрого преобразования Фурье (БПФ). Практическая реализация и тестирование различных подходов к вычислению ДПФ и БПФ подтвердили высокую точность этих методов. Особое внимание было уделено сравнению производительности, что в очередной раз продемонстрировало значительное преимущество БПФ при обработке сигналов большой длины. Также были отмечены нюансы и сравнительные характеристики рекурсивных и итеративных (нерекурсивных) реализаций БПФ в контексте различных объемов входных данных и возможных ограничений стека вызовов.